In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mphelper import ProcessWrapPool
from Bio import SeqIO
import matplotlib
from matplotlib.colors import LinearSegmentedColormap
import itertools
from biodatatools.utils.common import json_dump
from biodata.bed import BEDXReader
from collections import defaultdict
import sys
from pathlib import Path

In [2]:
matplotlib.font_manager.fontManager.addfont(f"{Path.cwd().parent}/font/Arial.ttf")
matplotlib.rcParams["font.family"] = "Arial"
bfontsize = 12
sfontsize = 9

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [3]:
sys.path.append(f"{Path.cwd().parent}")
import utils

In [4]:
PROJECT_DIR_d = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/"
PROJECT_DIR_r = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/resources/"
PROJECT_DIR_o = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/output/"

# Get sequences

In [5]:
s = "C1"
ds = ["distal", "proximal"]
ps = ["divergent", "unidirectional"]

In [ ]:
inputfile = f"{PROJECT_DIR_r}genomes/human/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta"
with open(inputfile, "r") as f:
	fdict = SeqIO.to_dict(SeqIO.parse(f, "fasta"))

In [ ]:
# 100bp region
# Divergent: anchor on maximum and minimum TSSs, separately
# Unidirectional: anchor on one prominent TSSs

def process_sequence(fdict, chrom, position, strand, tsstype, extension=50):
	seq = fdict[chrom][int(position)-extension:int(position)+extension+1].seq
	if (tsstype == "minTSS" and strand == "+") or (tsstype != "minTSS" and strand == "-"):
		seq = seq.reverse_complement()
	return str(seq)

In [ ]:
seqs = defaultdict(list)
for p, d in itertools.product(ps, ds):
	inputfile = f"{PROJECT_DIR_d}bed_plot/{s}_{p}_{d}.bed"
	if p == "unidirectional":
		with BEDXReader(inputfile, ["summit_position", "na", "strand"], x=5) as br:
			for r in br:
				seq = process_sequence(fdict, r.chrom, r.summit_position, r.strand, "uniTSS")
				seqs[f"{p.capitalize()}_{d}"].append(seq)
	else:
		with BEDXReader(inputfile, ["revTSS", "fwdTSS", "strand"], x=5) as br:
			for r in br:
				if r.strand == "+":
					seq_max = process_sequence(fdict, r.chrom, r.fwdTSS, r.strand, "maxTSS")
					seq_min = process_sequence(fdict, r.chrom, r.revTSS, r.strand, "minTSS")
				else:
					seq_max = process_sequence(fdict, r.chrom, r.revTSS, r.strand, "maxTSS")
					seq_min = process_sequence(fdict, r.chrom, r.fwdTSS, r.strand, "minTSS")
				seqs[f"{p.capitalize()}_maxTSS_{d}"].append(seq_max)
				seqs[f"{p.capitalize()}_minTSS_{d}"].append(seq_min)

In [9]:
# Also used in "1-3.Fig2c.Core_promoter_motifs.ipynb"

json_dump(f"{PROJECT_DIR_o}temp/{s}_TSS_seqs.json", seqs)

# Get 3-mer counts

In [ ]:
nucs = ["A", "T", "C", "G"]
kmers = ["".join(kmer) for kmer in itertools.product(nucs, repeat=3)]
len(kmers)

In [ ]:
# Check position-specific 3-mer counts normalized to total 3-mer frequencies for a given category

df_heatmaps = {}
for k in seqs:
	tot = defaultdict(int)
	pos = {kmer: [0] * len(seqs[k][0]) for kmer in kmers}
	# 1bp step, 3bp window
	for seq in seqs[k]:
		for n in range(len(seq) - 2):
			ss = seq[n:n+3]
			tot[ss] += 1
			for n2 in range(n, n+3):
				pos[ss][n2] += 1
	results = []
	for kmer in kmers:
		results.append([k/tot[kmer] for k in pos[kmer]])
	df_heatmaps[k] = pd.DataFrame(results, index=kmers)

# Heatmaps

## Settings

In [ ]:
ms = ["Divergent_maxTSS", "Divergent_minTSS", "Unidirectional"]

In [13]:
for d in ds:
	maxs = []
	mins = []
	for m in ms:
		maxs.append(max(df_heatmaps[f"{m}_{d}"].max()))
		mins.append(min(df_heatmaps[f"{m}_{d}"].min()))
	print(d, max(maxs), min(mins))

distal 0.133954857703631 0.002536556251865115
proximal 0.15913200723327306 0.0008807985907222549


In [ ]:
cvals = [0, 0.2]
colors = ["#ffffff", "#d7191c"]
norm=plt.Normalize(min(cvals), max(cvals))
tuples = list(zip(map(norm, cvals), colors))
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", tuples)
cmap

## Generate heatmaps

In [ ]:
def generate_heatmap(d, ms, df_heatmaps, cbar_ticks, cmap, outputfile, test, xlabel="Distance (bp)"):
	fig, axes = plt.subplots(1, len(ms), figsize=(4.5, 8.5))
	labelpad = 2
	wspace = 0.2
	
	for m in ms:
		col = ms.index(m)
		ax = axes[col]

		if col == len(ms)-1:
			cbar = True
			cbar_ax = fig.add_axes([0.93, 0.77, 0.03, 0.1])
		else:
			cbar = False
			cbar_ax = None						
		cbar_kws = {"ticks": cbar_ticks}

		yticklabels = True if col == 0 else False
		utils.generate_feature_heatmap(df_heatmaps[(f"{m}_{d}")], cbar_ticks, cmap, cbar, cbar_ax, cbar_kws, ax, None, test, yticklabels)
		if col == 0:
			ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=7)
		ax.set_ylabel("")
		
		if col == len(ms)-1:
			cbar_ax.set_yticklabels(cbar_ticks)
			cbar_ax.tick_params(axis="y", labelsize=sfontsize, pad=labelpad)
		
		if "_" in m:
			title = f"{m.split('_')[0]}\n({m.split('_')[1]})"
			pad = labelpad+5
		else:
			title = m
			pad = labelpad+10
		ax.set_title(title, fontsize=bfontsize, pad=pad)
			
		xtick_list = [0, 25, 50, 75, 100]
		xticklabel_list = ["-50", "-25", "0", "25", "50"]
		ax.set_xlim([0, 100])
		ax.set_xticks(xtick_list)
		ax.set_xticklabels(xticklabel_list)
		ax.set_xlabel(xlabel, fontsize=bfontsize)
		
		ax.tick_params(labelsize=sfontsize, pad=labelpad)

	fig.subplots_adjust(wspace=wspace)
	plt.savefig(outputfile, bbox_inches="tight", transparent=True)

In [ ]:
pwpool = ProcessWrapPool(len(ds))

# test = True
test = False
cbar_ticks = [0, 0.1, 0.2]
for d in ds:
	folder = "figures/" if d == "distal" else "other_figures/"
	outputfile = f"{PROJECT_DIR_o}{folder}Fig2b.pdf"
	pwpool.run(generate_heatmap, args=[d, ms, df_heatmaps, cbar_ticks, cmap, outputfile, test])

In [ ]:
len(pwpool.finished_tasks)

In [19]:
pwpool.close()